# Minimizing RAM Usage with Batch Processing and Optimizing Datatypes
This example project illustrates the processes of determining optimal batch size for processing with chunk iterators (TextFileReader) in pandas with use of converters and dtype parameters for pd.read_csv() for optimal typecasting.  The assumption is that only 10 MB of RAM is available for any subsequent analysis, so the objective is to create a plan for loading the data accordingly.

## Import Necessary Libraries

In [1]:
%load_ext nb_black

import pandas as pd
import numpy as np

pd.options.display.max_columns = 99

<IPython.core.display.Javascript object>

## Load and Examine First 5 Rows of csv File

In [2]:
first_5 = pd.read_csv("loans_2007.csv", nrows=5)
print(first_5.shape[1], "columns")
first_5

52 columns


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

#### Observations:
- There are 52 columns.
- `id` is nominal.
- There are several float type columns with trailing zeros after the decimal, that might be cast as integers if they don't have missing values.  However, as they were cast as floats, some null values are likely.
- There are some apparent categorical columns, such as `term`.
- `int_rate` and `revol_util` will be of object type, but can be cleaned and converted to float.
- `emp_title` can be ignored.
- `issue_d`, `earliest_cr_line`, `last_payment_d`, and `last_credit_pull_d` can be cleaned and cast as dates.
- `pymnt_plan` can be cast as boolean.
- `title` may not be useful, but requires further exploration.
- `policy_code` may not be useful for analysis.
- `zip code` is represented by only its first 3 digits.  As we have the more granular `addr_state`, we will ignore `zip code`.

### Determine Appropriate Batch Size to Stay Under 5 MB

In [3]:
first_1000 = pd.read_csv("loans_2007.csv", nrows=1000)
first_1000.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          1000 non-null   int64  
 1   member_id                   1000 non-null   float64
 2   loan_amnt                   1000 non-null   float64
 3   funded_amnt                 1000 non-null   float64
 4   funded_amnt_inv             1000 non-null   float64
 5   term                        1000 non-null   object 
 6   int_rate                    1000 non-null   object 
 7   installment                 1000 non-null   float64
 8   grade                       1000 non-null   object 
 9   sub_grade                   1000 non-null   object 
 10  emp_title                   949 non-null    object 
 11  emp_length                  983 non-null    object 
 12  home_ownership              1000 non-null   object 
 13  annual_inc                  1000 n

<IPython.core.display.Javascript object>

#### Observations:
- We are tasked with running the analysis in batches, using less than 10 MB of memory.
- To be safe, we will aim to stay under 5 MB with each chunk.
- As 1000 rows uses ~1.6 MB, we will check 3000 rows.

#### Testing `chunksize=3000`

In [4]:
print("Memory Usage by Chunk in MB")
chunk_iter = pd.read_csv("loans_2007.csv", chunksize=3000)
[chunk.memory_usage(deep=True).sum() / (2**20) for chunk in chunk_iter]

Memory Usage by Chunk in MB


[4.649059295654297,
 4.644805908203125,
 4.646563529968262,
 4.647915840148926,
 4.644108772277832,
 4.645991325378418,
 4.644582748413086,
 4.646951675415039,
 4.645077705383301,
 4.64512825012207,
 4.657840728759766,
 4.656707763671875,
 4.663515090942383,
 4.896956443786621,
 0.880854606628418]

<IPython.core.display.Javascript object>

#### Observations:
- 3000 is a good `chunksize` with all chunks using under 5 MB.

## Exploring Dataset by Each Chunk

#### Checking Total Columns

In [5]:
chunk_iter = pd.read_csv("loans_2007.csv", chunksize=3000)
total_rows = np.array([len(chunk) for chunk in chunk_iter]).sum()
print(f"There are {total_rows} total rows in the dataset.")

There are 42538 total rows in the dataset.


<IPython.core.display.Javascript object>

#### Checking Total Columns by Type

In [6]:
print('Object Columns for Each Chunk:\n')
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
print([len(chunk.select_dtypes(['object']).columns) for chunk in chunk_iter], '\n')

print('Numeric Columns for Each Chunk:')
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
[len(chunk.select_dtypes(['float64', 'int64']).columns) for chunk in chunk_iter]

Object Columns for Each Chunk:

[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22] 

Numeric Columns for Each Chunk:


[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30]

<IPython.core.display.Javascript object>

#### Observations:
- There is some variation in the number of object and numeric type columns between the chunks.
- We can examine what changed more closely.

#### Checking Consistency of Numeric or Object Types across Chunks

In [7]:
chunk_iter = pd.read_csv("loans_2007.csv", chunksize=3000)
chunk_dtypes = pd.concat([chunk.dtypes for chunk in chunk_iter])
overall_dtypes = chunk_dtypes.groupby(chunk_dtypes.index).value_counts()
print(overall_dtypes)

acc_now_delinq              float64    15
addr_state                  object     15
annual_inc                  float64    15
application_type            object     15
chargeoff_within_12_mths    float64    15
collection_recovery_fee     float64    15
collections_12_mths_ex_med  float64    15
delinq_2yrs                 float64    15
delinq_amnt                 float64    15
dti                         float64    15
earliest_cr_line            object     15
emp_length                  object     15
emp_title                   object     15
funded_amnt                 float64    15
funded_amnt_inv             float64    15
grade                       object     15
home_ownership              object     15
id                          int64      13
                            object      2
initial_list_status         object     15
inq_last_6mths              float64    15
installment                 float64    15
int_rate                    object     15
issue_d                     object

<IPython.core.display.Javascript object>

#### Observations:
- `id` is the column that changes type from numeric to object in the last 2 chunks.
- As a nominal feature it is not useful for analysis, so we can ignore it.

#### Creating Series of Overall Datatypes without `id`, `emp_title`, and `zip_code` from `overall_dtypes`

In [8]:
# Create series of overall datatypes without id and emp_title columns from overall_dtypes above
dtypes_keep_cols = (
    overall_dtypes.drop(["id", "emp_title", "zip_code"])
    .reset_index(level=[1])
    .drop(0, axis=1)["level_1"]
)
print(dtypes_keep_cols)

acc_now_delinq                float64
addr_state                     object
annual_inc                    float64
application_type               object
chargeoff_within_12_mths      float64
collection_recovery_fee       float64
collections_12_mths_ex_med    float64
delinq_2yrs                   float64
delinq_amnt                   float64
dti                           float64
earliest_cr_line               object
emp_length                     object
funded_amnt                   float64
funded_amnt_inv               float64
grade                          object
home_ownership                 object
initial_list_status            object
inq_last_6mths                float64
installment                   float64
int_rate                       object
issue_d                        object
last_credit_pull_d             object
last_pymnt_amnt               float64
last_pymnt_d                   object
loan_amnt                     float64
loan_status                    object
member_id   

<IPython.core.display.Javascript object>

## Initial Memory Footprint

In [9]:
print("Total Memory Usage in MB:\n")
chunk_iter = pd.read_csv(
    "loans_2007.csv", chunksize=3000, usecols=dtypes_keep_cols.index
)
chunk_memory = pd.concat([chunk.memory_usage(deep=True) for chunk in chunk_iter])
initial_memory_usage = chunk_memory.groupby(chunk_memory.index).sum() / (2**20)
print(initial_memory_usage)
print("")
print("Total: ", np.round(initial_memory_usage.sum(), 2), "MB")

Total Memory Usage in MB:

Index                         0.001884
acc_now_delinq                0.324539
addr_state                    2.393399
annual_inc                    0.324539
application_type              2.717916
chargeoff_within_12_mths      0.324539
collection_recovery_fee       0.324539
collections_12_mths_ex_med    0.324539
delinq_2yrs                   0.324539
delinq_amnt                   0.324539
dti                           0.324539
earliest_cr_line              2.635874
emp_length                    2.581555
funded_amnt                   0.324539
funded_amnt_inv               0.324539
grade                         2.677351
home_ownership                2.543880
initial_list_status           2.677351
inq_last_6mths                0.324539
installment                   0.324539
int_rate                      2.596222
issue_d                       2.636786
last_credit_pull_d            2.636661
last_pymnt_amnt               0.324539
last_pymnt_d                  2.63417

<IPython.core.display.Javascript object>

#### Observations:
- Our task now is to improve upon the total ~60 MB memory usage via typecasting columns appropriately.

#### Checking Number of Unique Values for Each Column for Potential Category Type

In [10]:
exclude_cols = [
    "int_rate",
    "revol_util",
    "issue_d",
    "earliest_cr_line",
    "last_pymnt_d",
    "last_credit_pull_d",
]
cat_cols = (
    dtypes_keep_cols[dtypes_keep_cols == "object"].drop(exclude_cols).index.tolist()
)

print("Number of Unique Values")
cat_candidates = []
for col in cat_cols:
    chunk_iter = pd.read_csv("loans_2007.csv", chunksize=3000, usecols=[col])
    chunk_vcs = pd.concat([chunk[col].value_counts() for chunk in chunk_iter])
    total_vcs = chunk_vcs.groupby(chunk_vcs.index).sum()
    print(col, ":", len(total_vcs))
    if len(total_vcs.index) / total_rows < 0.5:
        cat_candidates.append(col)
print(
    f"\nThere are {len(cat_candidates)} columns that have fewer than 50% of values are unique:\n"
)
print(cat_candidates)
col_types = {col: "category" for col in cat_candidates}

Number of Unique Values
addr_state : 50
application_type : 1
emp_length : 11
grade : 7
home_ownership : 5
initial_list_status : 1
loan_status : 9
purpose : 14
pymnt_plan : 2
sub_grade : 35
term : 2
title : 21264
verification_status : 3

There are 13 columns that have fewer than 50% of values are unique:

['addr_state', 'application_type', 'emp_length', 'grade', 'home_ownership', 'initial_list_status', 'loan_status', 'purpose', 'pymnt_plan', 'sub_grade', 'term', 'title', 'verification_status']


<IPython.core.display.Javascript object>

#### Observations:
- As suspected, `title` has numerous unique values and is not likely to add value to the analysis.  We will ignore it.
- The remaining object type columns with low unique values can be cast as category.

#### Excluding `title`

In [11]:
dtypes_keep_cols.drop("title", inplace=True)
cat_candidates.remove("title")

<IPython.core.display.Javascript object>

#### Creating Dictionary for `coltypes` for Use in `chunk_iter`

In [12]:
col_types = {col: "category" for col in cat_candidates}

<IPython.core.display.Javascript object>

#### Checking for Numerical Columns that can be Downcast

In [13]:
num_cols = dtypes_keep_cols[dtypes_keep_cols == "float64"].index.tolist()

print("Total Missing Values")
cols_to_int = []
for col in num_cols:
    chunk_iter = pd.read_csv("loans_2007.csv", chunksize=3000, usecols=[col])
    total_nulls = np.array([chunk[col].isna().sum() for chunk in chunk_iter]).sum()
    print(col, ": ", total_nulls)
    if total_nulls == 0:
        cols_to_int.append(col)
print("")
print(f"There are {len(cols_to_int)} remaining numeric columns without missing values.")

Total Missing Values
acc_now_delinq :  32
annual_inc :  7
chargeoff_within_12_mths :  148
collection_recovery_fee :  3
collections_12_mths_ex_med :  148
delinq_2yrs :  32
delinq_amnt :  32
dti :  3
funded_amnt :  3
funded_amnt_inv :  3
inq_last_6mths :  32
installment :  3
last_pymnt_amnt :  3
loan_amnt :  3
member_id :  3
open_acc :  32
out_prncp :  3
out_prncp_inv :  3
policy_code :  3
pub_rec :  32
pub_rec_bankruptcies :  1368
recoveries :  3
revol_bal :  3
tax_liens :  108
total_acc :  32
total_pymnt :  3
total_pymnt_inv :  3
total_rec_int :  3
total_rec_late_fee :  3
total_rec_prncp :  3

There are 0 remaining numeric columns without missing values.


<IPython.core.display.Javascript object>

#### Observations:
- All of the remaining numeric columns have missing values, so they will remain as float to accommodate NaNs.
- However, we can check to see if downcasting to a less memory-intensive float type is possible.

In [14]:
chunk_iter = pd.read_csv("loans_2007.csv", chunksize=3000)

mapping = {"float16": 0, "float32": 1, "float64": 2}
for col in num_cols:
    for chunk in chunk_iter:
        low_type = "float16"
        chunk[col] = pd.to_numeric(chunk[col], downcast="float")
        if mapping[str(chunk[col].dtype)] > mapping[low_type]:
            low_type = str(chunk[col].dtype)
    col_types[col] = low_type
col_types

{'addr_state': 'category',
 'application_type': 'category',
 'emp_length': 'category',
 'grade': 'category',
 'home_ownership': 'category',
 'initial_list_status': 'category',
 'loan_status': 'category',
 'purpose': 'category',
 'pymnt_plan': 'category',
 'sub_grade': 'category',
 'term': 'category',
 'verification_status': 'category',
 'acc_now_delinq': 'float32',
 'annual_inc': 'float32',
 'chargeoff_within_12_mths': 'float32',
 'collection_recovery_fee': 'float32',
 'collections_12_mths_ex_med': 'float32',
 'delinq_2yrs': 'float32',
 'delinq_amnt': 'float32',
 'dti': 'float32',
 'funded_amnt': 'float32',
 'funded_amnt_inv': 'float32',
 'inq_last_6mths': 'float32',
 'installment': 'float32',
 'last_pymnt_amnt': 'float32',
 'loan_amnt': 'float32',
 'member_id': 'float32',
 'open_acc': 'float32',
 'out_prncp': 'float32',
 'out_prncp_inv': 'float32',
 'policy_code': 'float32',
 'pub_rec': 'float32',
 'pub_rec_bankruptcies': 'float32',
 'recoveries': 'float32',
 'revol_bal': 'float32',
 

<IPython.core.display.Javascript object>

#### Observations:
- All of the float type columns will fit into float32 type, an improvement over float64.

## Addressing the Remaining Object Columns
- The remaining columns in `excluded_cols` include the dates and the float values with "%".
- We can create a dictionary of functions to convert them using the `converters` parameter of `pd.read_csv()'.

#### View the First 5 Rows

In [15]:
first_5[exclude_cols]

,int_rate,revol_util,issue_d,earliest_cr_line,last_pymnt_d,last_credit_pull_d
0,10.65%,83.7%,Dec-2011,Jan-1985,Jan-2015,Jun-2016
1,15.27%,9.4%,Dec-2011,Apr-1999,Apr-2013,Sep-2013
2,15.96%,98.5%,Dec-2011,Nov-2001,Jun-2014,Jun-2016
3,13.49%,21%,Dec-2011,Feb-1996,Jan-2015,Apr-2016
4,12.69%,53.9%,Dec-2011,Jan-1996,Jun-2016,Jun-2016


<IPython.core.display.Javascript object>

#### Converter Function for `int_rate` and `revol_util`

In [17]:
def str_to_float(x):
    if x != "":
        x = float(x.strip(" %"))
    else:
        x = np.nan
    return x

<IPython.core.display.Javascript object>

#### Defining `converters` Dictionary

In [68]:
converters = {
    "issue_d": lambda x: pd.to_datetime(x, format="%b-%Y", errors="coerce"),
    "earliest_cr_line": lambda x: pd.to_datetime(x, format="%b-%Y", errors="coerce"),
    "last_pymnt_d": lambda x: pd.to_datetime(x, format="%b-%Y", errors="coerce"),
    "last_credit_pull_d": lambda x: pd.to_datetime(x, format="%b-%Y", errors="coerce"),
    "int_rate": lambda x: str_to_float(x),
    "revol_util": lambda x: str_to_float(x),
}

<IPython.core.display.Javascript object>

#### Observations:
- As we are coercing the errors to NaT values for datetime columns, we should check that missing values are not excessive.

#### Checking Missing Values for datetime Columns

In [70]:
for col in ["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"]:
    chunk_iter = pd.read_csv(
        "loans_2007.csv",
        chunksize=3000,
        converters=converters,
        dtype=col_types,
        usecols=[col],
    )
    print([chunk[col].isna().sum() for chunk in chunk_iter])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 31]
[2, 5, 8, 6, 9, 4, 4, 1, 4, 6, 8, 9, 5, 13, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 4]


<IPython.core.display.Javascript object>

#### Observations:
- We have very few missing values for `issue_d`, `earliest_cr_line`, and `last_credit_pull_d`.
- There are a few more for `last_pymt_d`, which may be consistent with no payment yet received.
- These few entries can be further examined during analysis.

## Final Memory Usage after RAM Optimization

#### Verifying that All Columns Treated with either `converters` or `dtype`

In [79]:
# Checking that length of usecols is same as length of converters and col_types
usecols = dtypes_keep_cols.index.tolist()
print(len(usecols) == len(converters) + len(col_types))

# Checking that all columns in usecols are in converters + col_types
[
    item
    for item in usecols
    if item not in list(converters.keys()) + list(col_types.keys())
] == []

True


True

<IPython.core.display.Javascript object>

## Final Memory Footprint

In [99]:
print("Memory Footprint in MB:\n")
chunk_iter = pd.read_csv(
    "loans_2007.csv",
    chunksize=3000,
    usecols=usecols,
    converters=converters,
    dtype=col_types,
)
chunk_memory = pd.concat([chunk.memory_usage(deep=True) for chunk in chunk_iter])
final_memory_usage = chunk_memory.groupby(chunk_memory.index).sum() / (2**20)
print(final_memory_usage, "\n")
print("Total: ", final_memory_usage.sum(), "\n\n")
print(
    f"Total decrease in memory footprint with optimization: {np.round(initial_memory_usage.sum() - final_memory_usage.sum(), 2)} MB"
)

Memory Footprint in MB:

Index                         0.001884
acc_now_delinq                0.162270
addr_state                    0.096841
annual_inc                    0.162270
application_type              0.041526
chargeoff_within_12_mths      0.162270
collection_recovery_fee       0.162270
collections_12_mths_ex_med    0.162270
delinq_2yrs                   0.162270
delinq_amnt                   0.162270
dti                           0.162270
earliest_cr_line              0.324539
emp_length                    0.055244
funded_amnt                   0.162270
funded_amnt_inv               0.162270
grade                         0.051754
home_ownership                0.045242
initial_list_status           0.041512
inq_last_6mths                0.162270
installment                   0.162270
int_rate                      0.324539
issue_d                       0.324539
last_credit_pull_d            0.324539
last_pymnt_amnt               0.162270
last_pymnt_d                  0.324539


<IPython.core.display.Javascript object>

#### Conclusions:
- We were able to drastically improve the memory footprint by excluding unnecessary columns, cleaning and converting numeric columns disguised as object type, casting date columns as datetime type, and downcasting floats to less memory-intensive subtypes.

# Appendix:  Additional Related Code Chunks

In [88]:
np.iinfo("int64")

iinfo(min=-9223372036854775808, max=9223372036854775807, dtype=int64)

<IPython.core.display.Javascript object>

In [91]:
first_1000.size

52000

<IPython.core.display.Javascript object>

In [93]:
first_1000._data

BlockManager
Items: Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')
Axis 1: RangeIndex(start=0, stop=1000, step=1)

<IPython.core.display.Javascript object>

In [94]:
# Estimating MB without memory_usage = deep
first_1000_MB = (
    first_1000.size * 8 / (2**20)
)  # info memory_usage estimate without "deep"
first_1000_MB

0.396728515625

<IPython.core.display.Javascript object>

In [95]:
# Comparison to above estimate without memory_usage = 'deep'
first_1000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          1000 non-null   int64  
 1   member_id                   1000 non-null   float64
 2   loan_amnt                   1000 non-null   float64
 3   funded_amnt                 1000 non-null   float64
 4   funded_amnt_inv             1000 non-null   float64
 5   term                        1000 non-null   object 
 6   int_rate                    1000 non-null   object 
 7   installment                 1000 non-null   float64
 8   grade                       1000 non-null   object 
 9   sub_grade                   1000 non-null   object 
 10  emp_title                   949 non-null    object 
 11  emp_length                  983 non-null    object 
 12  home_ownership              1000 non-null   object 
 13  annual_inc                  1000 n

<IPython.core.display.Javascript object>

In [96]:
# Examples of numpy.iinfo() to obtain min and max values type can represent
print(np.iinfo("int16"))
print(np.iinfo("int8").min)

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

-128


<IPython.core.display.Javascript object>

In [98]:
# Function to downcast for integer
def change_to_int(dataframe, col_name):
    col_max = dataframe[col_name].max()
    col_min = dataframe[col_name].min()
    for dtype_name in ["int8", "int16", "int32", "int64"]:
        if col_max < np.iint(dtype_name).max and col_min > np.iint(dtype_name).min:
            col_types[col] = dtype_name
            break

<IPython.core.display.Javascript object>